# 데이터 불러오기

In [3]:
import pandas as pd

df = pd.read_excel('../datasets/환율데이터.xlsx', index_col=0)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2459 entries, 1 to 2459
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   1Y_대비_irs  2459 non-null   float64
 1   1Y_대비_crs  2459 non-null   float64
 2   대비_국고10년   2459 non-null   float64
 3   대비_ndf     2459 non-null   float64
 4   lable      2459 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 115.3 KB


In [5]:
df.head(10)

,1Y_대비_irs,1Y_대비_crs,대비_국고10년,대비_ndf,lable
열1,,,,,
1,-0.03,-0.01,-0.04,1.75,0
2,-0.03,-0.01,-0.07,4.00,0
3,0.02,0.00,0.00,-9.50,0
4,0.01,0.02,0.00,0.50,1
5,0.01,0.01,-0.02,1.25,1
6,0.05,0.09,0.05,2.25,0
7,-0.03,0.01,-0.02,-2.50,0
8,0.00,0.00,0.02,1.00,1
9,0.05,0.04,0.05,3.00,0


# 학습 & 평가

In [6]:
X = df.drop('lable', axis=1)
y = df[['lable']]

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)

## 로지스틱 회귀

In [8]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(X_train, y_train)

pred = lr.predict(X_test)

from sklearn.metrics import confusion_matrix

confusion = confusion_matrix(y_test, pred)
confusion

c:\Users\qkrtj\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([[167,  61],
       [ 75, 189]], dtype=int64)

In [9]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, pred)
accuracy

0.7235772357723578

## 의사결정나무

In [10]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=156)

dt.fit(X_train, y_train)

pred = dt.predict(X_test)

from sklearn.metrics import confusion_matrix

confusion = confusion_matrix(y_test, pred)
confusion

array([[208,  20],
       [ 23, 241]], dtype=int64)

In [11]:
accuracy = accuracy_score(y_test, pred)
accuracy

0.9126016260162602

# 임계값 설정 - 0.6

In [12]:
pred_proba = lr.predict_proba(X_test)
pred = lr.predict(X_test)

import numpy as np
pred_proba_result = np.concatenate([pred_proba, pred.reshape(-1,1)],axis=1)
pred_proba_result

array([[0.30809876, 0.69190124, 1.        ],
       [0.92601006, 0.07398994, 0.        ],
       [0.2447083 , 0.7552917 , 1.        ],
       ...,
       [0.83563018, 0.16436982, 0.        ],
       [0.45415997, 0.54584003, 1.        ],
       [0.12957035, 0.87042965, 1.        ]])

In [13]:
from sklearn.preprocessing import Binarizer

custom_threshold = 0.6

pred_proba_1 = pred_proba[:,1].reshape(-1,1)

binarizer = Binarizer(threshold=custom_threshold).fit(pred_proba_1)
custom_predict = binarizer.transform(pred_proba_1)

accuracy = accuracy_score(y_test, custom_predict)
accuracy

0.7052845528455285

# GridSearchCV

In [15]:
from sklearn.model_selection import GridSearchCV

parameters = {"max_depth":[2,3,5,9,10],'min_samples_split':[2,3,5,7,9], 
              'min_samples_leaf':[1,5,8,9,10]}

grid_dclf = GridSearchCV(dt, param_grid = parameters, scoring='accuracy',cv=5)
grid_dclf.fit(X_train, y_train)
print(grid_dclf.best_params_)
print(grid_dclf.best_score_)
best_dclf = grid_dclf.best_estimator_

dpredictions = best_dclf.predict(X_test)
accuracy = accuracy_score(y_test, dpredictions)
accuracy

{'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 3}
0.9125624830472352


0.9024390243902439